# Klein-Gordon Equation

This notebook contains the programmatic verification for the **Klein-Gordon Equation** entry from the THEORIA dataset.

**Entry ID:** klein_gordon_equation  
**Required Library:** sympy 1.13.1

## Description
The Klein-Gordon equation is the relativistic wave equation for spin-0 (scalar) particles. It arises from applying quantum mechanical operator substitutions to the relativistic energy-momentum relation `E^2 = (p*c)^2 + (m*c^2)^2`. Unlike the Schrödinger equation, it is second-order in time, which initially caused difficulties with probability interpretation but is now understood through quantum field theory. The equation describes scalar mesons (pions, kaons) and the Higgs boson.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# =====================================================
# Programmatic verification: Klein-Gordon Equation
#
# Sections:
#  1. Setup: Define symbols and wave function
#  2. Verify derivation Steps 4-8: Build KG equation
#  3. Verify Steps 9-10: d'Alembertian equivalence
#  4. Plane-wave solution: Verify dispersion relation
# =====================================================

# ---------------------------
# Section 1: Setup
# ---------------------------
t, x, y, z = sp.symbols('t x y z', real=True)
m, c, hbar = sp.symbols('m c hbar', positive=True, real=True)
psi = sp.Function('psi')(t, x, y, z)

# ---------------------------
# Section 2: Steps 4-8 - Build Klein-Gordon equation
# ---------------------------

# Step 4: Squared energy operator
# E^2 psi = (i*hbar*d/dt)^2 psi = -hbar^2 * d^2psi/dt^2
E2_psi = -hbar**2 * sp.diff(psi, t, 2)

# Step 5: Squared momentum operator (Laplacian)
# p^2 psi = (-i*hbar*nabla)^2 psi = -hbar^2 * nabla^2 psi
laplacian_psi = sp.diff(psi, x, 2) + sp.diff(psi, y, 2) + sp.diff(psi, z, 2)
p2_psi = -hbar**2 * laplacian_psi

# Step 6: Substitute into E^2 = (pc)^2 + (mc^2)^2
# -hbar^2 * d^2psi/dt^2 = -hbar^2 * c^2 * nabla^2 psi + (mc^2)^2 * psi
lhs_step6 = E2_psi
rhs_step6 = c**2 * p2_psi + (m*c**2)**2 * psi

# Verify step 6 equation holds symbolically
step6_relation = sp.Eq(lhs_step6, rhs_step6)

# Steps 7-8: Rearrange to standard form
# (1/c^2)*d^2psi/dt^2 - nabla^2 psi + (m^2 c^2 / hbar^2)*psi = 0
kg_term1 = (1/c**2) * sp.diff(psi, t, 2)  # Time term
kg_term2 = -laplacian_psi                  # Spatial term
kg_term3 = (m**2 * c**2 / hbar**2) * psi   # Mass term
kg_expanded = kg_term1 + kg_term2 + kg_term3

# ---------------------------
# Section 3: Steps 9-10 - d'Alembertian form
# ---------------------------

# Step 9: d'Alembertian operator applied to psi
# square psi = (1/c^2)*d^2psi/dt^2 - nabla^2 psi
box_psi = (1/c**2) * sp.diff(psi, t, 2) - laplacian_psi

# Step 10: KG in d'Alembertian form
# (square + (mc/hbar)^2) psi = 0
kg_compact = box_psi + (m*c/hbar)**2 * psi

# Verify equivalence of expanded and compact forms
assert sp.simplify(kg_expanded - kg_compact) == 0, 'Expanded and compact forms must be equivalent'

# ---------------------------
# Section 4: Plane-wave solution verification
# ---------------------------

# Define plane wave: psi = exp(i*(-omega*t + kx*x + ky*y + kz*z))
omega, kx, ky, kz = sp.symbols('omega k_x k_y k_z', real=True)
E_pw, px, py, pz = sp.symbols('E p_x p_y p_z', real=True)

# Phase: phi = -omega*t + k.r
phase = sp.I * (-omega*t + kx*x + ky*y + kz*z)
plane_wave = sp.exp(phase)

# Apply KG operator to plane wave
# d^2/dt^2 of exp(i*(-omega*t + ...)) = (-i*omega)^2 * exp(...) = -omega^2 * exp(...)
d2_dt2_pw = sp.diff(plane_wave, t, 2)
assert sp.simplify(d2_dt2_pw + omega**2 * plane_wave) == 0, 'Time derivative check'

# nabla^2 of plane wave = (i*k)^2 * exp(...) = -k^2 * exp(...)
laplacian_pw = sp.diff(plane_wave, x, 2) + sp.diff(plane_wave, y, 2) + sp.diff(plane_wave, z, 2)
k_squared = kx**2 + ky**2 + kz**2
assert sp.simplify(laplacian_pw + k_squared * plane_wave) == 0, 'Laplacian check'

# KG equation for plane wave:
# (1/c^2)*(-omega^2) - (-k^2) + (mc/hbar)^2 = 0
# => -omega^2/c^2 + k^2 + (mc/hbar)^2 = 0
# => omega^2/c^2 = k^2 + (mc/hbar)^2
# => omega^2 = c^2*k^2 + (mc^2/hbar)^2

kg_plane_wave = kg_expanded.subs(psi, plane_wave)
kg_simplified = sp.simplify(sp.expand(kg_plane_wave / plane_wave))

# The dispersion relation from KG equation
dispersion_from_kg = sp.simplify(kg_simplified * c**2)
# Should be: -omega^2 + c^2*k^2 + (mc^2/hbar)^2 = 0 times some factor

# Verify dispersion relation matches energy-momentum relation
# Using E = hbar*omega, p = hbar*k:
# E^2 = (hbar*omega)^2 = hbar^2 * omega^2
# p^2 = (hbar*k)^2 = hbar^2 * k^2
# E^2 = (pc)^2 + (mc^2)^2
# => hbar^2*omega^2 = hbar^2*c^2*k^2 + (mc^2)^2
# => omega^2 = c^2*k^2 + (mc^2/hbar)^2

expected_dispersion = -omega**2/c**2 + k_squared + (m*c/hbar)**2
assert sp.simplify(kg_simplified - expected_dispersion) == 0, 'Dispersion relation must match'

# Verify this is consistent with E^2 = (pc)^2 + (mc^2)^2
# Substitute E = hbar*omega, |p|^2 = hbar^2*k^2
E_from_omega = hbar * omega
p2_from_k = hbar**2 * k_squared

# Check: E^2 - (pc)^2 - (mc^2)^2 = 0 when dispersion holds
# omega^2 = c^2*k^2 + (mc^2/hbar)^2 => multiply by hbar^2:
# hbar^2*omega^2 = hbar^2*c^2*k^2 + (mc^2)^2
# E^2 = p^2*c^2 + (mc^2)^2 ✓

energy_momentum_check = E_from_omega**2 - p2_from_k * c**2 - (m*c**2)**2
# When the KG dispersion holds: omega^2 = c^2*k^2 + (mc^2/hbar)^2
# => hbar^2*omega^2 - hbar^2*c^2*k^2 - (mc^2)^2 = 0
em_substituted = energy_momentum_check.subs(omega**2, c**2*k_squared + (m*c**2/hbar)**2)
assert sp.simplify(em_substituted) == 0, 'Energy-momentum relation must be satisfied'

print('All Klein-Gordon equation verifications passed')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=klein_gordon_equation.json](https://theoria-dataset.org/entries.html?entry=klein_gordon_equation.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0